In [1]:
from astropy.table import Table, Column, Row
import random
import time
from datetime import date
import calendar
import numpy as np
import matplotlib.pyplot as plt

First I read in my weather stats table as provided by Jess

In [2]:
wdata = Table.read('data/weather-average.csv', format='csv')

# wdata = Table.read('data/weather-average-dry.csv', format='csv')
# wdata = Table.read('weather-average-wet.csv', format='csv')

I then make new columns containing the cumulative fraction of time in each weather band (for use later). 

In [3]:
CumulativeBand1 = Column(wdata['Band1Percentage'], name='CumulativeBand1', format='.1f')
wdata.add_column(CumulativeBand1)
CumulativeBand2 = Column([x['CumulativeBand1']+x['Band2Percentage'] for x in wdata], name='CumulativeBand2', format='.1f')
wdata.add_column(CumulativeBand2)
CumulativeBand3 = Column([x['CumulativeBand2']+x['Band3Percentage'] for x in wdata], name='CumulativeBand3', format='.1f')
wdata.add_column(CumulativeBand3)
CumulativeBand4 = Column([x['CumulativeBand3']+x['Band4Percentage'] for x in wdata], name='CumulativeBand4', format='.1f')
wdata.add_column(CumulativeBand4)
CumulativeBand5 = Column([x['CumulativeBand4']+x['Band5Percentage'] for x in wdata], name='CumulativeBand5', format='.1f')
wdata.add_column(CumulativeBand5)

In [4]:
wdata

Month,Band1Percentage,Band2Percentage,Band3Percentage,Band4Percentage,Band5Percentage,CumulativeBand1,CumulativeBand2,CumulativeBand3,CumulativeBand4,CumulativeBand5
str9,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
February,15.7,23.3,16.0,20.7,17.8,15.7,39.0,55.0,75.7,93.5
March,23.5,23.1,15.8,21.0,8.8,23.5,46.6,62.4,83.4,92.2
April,16.2,20.0,25.2,19.4,12.4,16.2,36.2,61.4,80.8,93.2
May,10.6,30.4,27.8,21.2,7.8,10.6,41.0,68.8,90.0,97.8
June,6.2,28.6,30.3,25.6,7.1,6.2,34.8,65.1,90.7,97.8
July,3.2,27.6,32.3,22.2,9.2,3.2,30.8,63.1,85.3,94.5
August,6.9,27.6,27.7,15.7,12.9,6.9,34.5,62.2,77.9,90.8
September,1.4,34.7,34.6,16.6,5.2,1.4,36.1,70.7,87.3,92.5
October,5.5,14.8,29.1,28.2,12.9,5.5,20.3,49.4,77.6,90.5


I define a loook up dictionary for easy table access - months are not month values but rather positions in the table


In [5]:
dict = {'February': 0, 'March': 1, 'April': 2, 'May': 3, 'June': 4, 'July': 5, 'August': 6, 'September': 7, 'October': 8, 'November': 9, 'December': 10, 'January': 11}

Now I need to load in my schedule file from this it will detemrine the month and number of days I need to predict for

In [37]:
# need to supply table with list of dates on which we will observe - 
# from this table my code needs to know month and number of days - to allocated a weather band for the night
# the UT date and weather bad is then needed as input into Michael Polhen's code

sdata = Table.read('data/18A_schedule.csv', format='csv')

# splitting up YYYYMMDD into YYYY MM DD
yearForTable = Column([int(x/10000) for x in sdata['Date']], name='Year')
monthForTable = Column([calendar.month_name[ int(int(x-((int(x/10000))*10000))/100)] for x in sdata['Date']], name='Month')
dayForTable = Column([x - (int(x/100)*100) for x in sdata['Date']], name='Day')
sdata.add_column(yearForTable)
sdata.add_column(monthForTable)
sdata.add_column(dayForTable)

# obtaines data when not PI time (PI = UH and DDT and E&C)

LAPsdata = sdata[sdata['Priority1']!='PI'] 

# for now only look at one month

month = 'March' #### CHANGE

MonthLAPSdata = LAPsdata[LAPsdata['Month']==month]

numberofdays = (len(MonthLAPSdata))

MonthLAPSdata

Date,Priority1,Priority2,Year,Month,Day
int64,str8,str8,int64,str8,int64
20180302,M16AL006,M17BL009,2018,March,2
20180303,M16AL006,M17BL009,2018,March,3
20180304,M16AL006,M17BL009,2018,March,4
20180305,M16AL006,M17BL009,2018,March,5
20180309,M17BL002,--,2018,March,9
20180310,M17BL002,--,2018,March,10
20180311,M17BL002,--,2018,March,11
20180312,M17BL002,--,2018,March,12
20180313,M17BL002,--,2018,March,13


In [38]:
print ("Examining {} nights alloacted in {} \n".format(numberofdays,month))
print (("fraction of nights Band 1={}%, 2={}%, 3={}%, 4={}% and 5={}%\n".format(wdata['Band1Percentage'][dict[month]],wdata['Band2Percentage'][dict[month]],wdata['Band3Percentage'][dict[month]],wdata['Band4Percentage'][dict[month]],wdata['Band5Percentage'][dict[month]])))


Examining 19 nights alloacted in March 

fraction of nights Band 1=23.5%, 2=23.1%, 3=15.8%, 4=21.0% and 5=8.8%



In [62]:
# keeping it simple - allocating an entire night to one weather band
# ut doing a crappy job - issue is with small number of nights

b1nights = round(numberofdays*wdata['Band1Percentage'][dict[month]]/100,)
b2nights = round(numberofdays*wdata['Band2Percentage'][dict[month]]/100)
b3nights = round(numberofdays*wdata['Band3Percentage'][dict[month]]/100)
b4nights = round(numberofdays*wdata['Band4Percentage'][dict[month]]/100)
b5nights = round(numberofdays*wdata['Band5Percentage'][dict[month]]/100)
b6nights = round(numberofdays*(100-wdata['CumulativeBand5'][dict[month]])/100)

#print((numberofdays*(100-wdata['CumulativeBand5'][dict[month]])/100))

#print (b1nights,b2nights,b3nights,b4nights,b5nights,b6nights)

sdatabanddata  = []

for i in range(numberofdays):
#     print(i)
    if 0 <= i < b1nights:
#         print ("band 1")
        sdatabanddata.append(1)
    if b1nights <= i < (b1nights+b2nights):
#         print ("band 2")
        sdatabanddata.append(2)
    if (b1nights+b2nights) <= i < (b1nights+b2nights+b3nights):
#         print ("band 3")
        sdatabanddata.append(3)
    if (b1nights+b2nights+b3nights) <= i < (b1nights+b2nights+b3nights+b4nights):
#         print ("band 4")
        sdatabanddata.append(4)
    if (b1nights+b2nights+b3nights+b4nights) <= i < (b1nights+b2nights+b3nights+b4nights+b5nights):
#         print ("band 5")
        sdatabanddata.append(5)
    if (b1nights+b2nights+b3nights+b4nights+b5nights) <= i:
#         print ("closed")
        sdatabanddata.append(6)

print (sdatabanddata,type(sdatabanddata))
myarray = np.asarray(sdatabanddata)
print (myarray,type(myarray))

print (random.shuffle(sdatabanddata))
print (random.shuffle(myarray))

[1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5, 6, 6] <class 'list'>
[1 1 1 1 2 2 2 2 3 3 3 4 4 4 4 5 5 6 6] <class 'numpy.ndarray'>
None
None


I then generate random numbers from 0-100 for the number of days dedicate to LAP programs in my month and use the "cumulative fraction of weather" stats to allocated an entire night to a weather band

In [17]:

print (("cumulative nights Band 1={}%, 1-2={}%, 1-3={}%, 1-4={}% and 1-5={}%\n".format(wdata['CumulativeBand1'][dict[month]],wdata['CumulativeBand2'][dict[month]],wdata['CumulativeBand3'][dict[month]],wdata['CumulativeBand4'][dict[month]],wdata['CumulativeBand5'][dict[month]])))


b1 = 0 # counter for number of days in band 1
b2 = 0
b3 = 0
b4 = 0
b5 = 0
b6 = 0
sdatabanddata  = ([])
randomweatherdata = ([])
for i in range(0,10000):

    randomweather = random.sample(range(1, 100),numberofdays)

    for x in randomweather:
        #print (x)
        randomweatherdata.append(x)
        if x <= wdata['CumulativeBand1'][dict[month]]:
            #print ("Band 1 night")
            b1 += 1
            sdatabanddata.append(1)
        if wdata['CumulativeBand1'][dict[month]] < x <= wdata['CumulativeBand2'][dict[month]]: 
            #print ("Band 2 night")
            b2 += 1
            sdatabanddata.append(2)
        if wdata['CumulativeBand2'][dict[month]] < x <= wdata['CumulativeBand3'][dict[month]]:  
            #print ("Band 3 night")
            b3 += 1
            sdatabanddata.append(3)
        if wdata['CumulativeBand3'][dict[month]] < x <= wdata['CumulativeBand4'][dict[month]]:  
            #print ("Band 4 night")
            b4 += 1
            sdatabanddata.append(4)
        if wdata['CumulativeBand4'][dict[month]] < x <= wdata['CumulativeBand5'][dict[month]]: 
            #print ("Band 5 night")
            b5 += 1
            sdatabanddata.append(5)
        if x >= wdata['CumulativeBand5'][dict[month]]:
            #print ("Closed")
            b6 += 1
            sdatabanddata.append(6)
            
    #print ("\nfraction output 1={:.2f}%, 2={:.2f}%, 3={:.2f}%, 4={:.2f}% and 5={:.2f}% (closed {:.2f}%)".format(b1/numberofdays*100,b2/numberofdays*100,b3/numberofdays*100,b4/numberofdays*100,b5/numberofdays*100,b6/numberofdays*100))

#print (sdatabanddata)
print (sdatabanddata.count(1)/len(sdatabanddata))
print (sdatabanddata.count(2)/len(sdatabanddata))
print (sdatabanddata.count(3)/len(sdatabanddata))
print (sdatabanddata.count(4)/len(sdatabanddata))
print (sdatabanddata.count(5)/len(sdatabanddata))
print (sdatabanddata.count(6)/len(sdatabanddata))


Examining 14 nights alloacted in February 

fraction of nights Band 1=15.7%, 2=23.3%, 3=16.0%, 4=20.7% and 5=17.8%

cumulative nights Band 1=15.7%, 1-2=39.0%, 1-3=55.0%, 1-4=75.7% and 1-5=93.5%

0.15182857142857142
0.24171428571428571
0.16129285714285715
0.20086428571428572
0.18273571428571428
0.06156428571428572


In [ ]:
plt.hist(sdatabanddata, bins=[1,2,3,4,5,6,7], edgecolor='white', facecolor='orange', alpha=0.8, normed=True)
plt.axhline(sdatabanddata.count(1)/len(sdatabanddata), color='blue', linestyle='dashed', linewidth=1)
plt.axhline(sdatabanddata.count(2)/len(sdatabanddata), color='gray', linestyle='dashed', linewidth=1)
plt.axhline(sdatabanddata.count(3)/len(sdatabanddata), color='gray', linestyle='dashed', linewidth=1)
plt.axhline(sdatabanddata.count(4)/len(sdatabanddata), color='gray', linestyle='dashed', linewidth=1)
plt.axhline(sdatabanddata.count(5)/len(sdatabanddata), color='gray', linestyle='dashed', linewidth=1)
plt.axhline(sdatabanddata.count(6)/len(sdatabanddata), color='green', linestyle='dashed', linewidth=1)
plt.show()

print ((wdata['Band1Percentage'][dict[month]]/100),(wdata['Band2Percentage'][dict[month]]/100))
#print (sdatabanddata)


In [ ]:
plt.hist(randomweatherdata, edgecolor='white', facecolor='orange', alpha=0.8, normed=1)
plt.axvline((wdata['CumulativeBand1'][dict[month]]), color='gray', linestyle='dashed', linewidth=1)
plt.axvline((wdata['CumulativeBand2'][dict[month]]), color='gray', linestyle='dashed', linewidth=1)
plt.axvline((wdata['CumulativeBand3'][dict[month]]), color='gray', linestyle='dashed', linewidth=1)
plt.axvline((wdata['CumulativeBand4'][dict[month]]), color='gray', linestyle='dashed', linewidth=1)
plt.axvline((wdata['CumulativeBand5'][dict[month]]), color='gray', linestyle='dashed', linewidth=1)
plt.show()

In [ ]:
#sdatabanddata.T
#print (sdatabanddata.T)
#banddata = Column(sdatabanddata, name='WeatherBand')
#sdata.add_column(banddata)


In essence I then need to calculate how many night in a month are in each weatherbands and run Michael's code on the relevent 